In [ ]:
!pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 31.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 70 kB 1.6 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629484 sha256=43a460ed448c8e52c280a5117a04176e9f14978a0cf71ded3c9eb857ad4be010
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65034 sha256=17421d72c2d522de11ba38a915d1530b9de670c085745ba12242e0974d803e06
  Stored in directory: /root/.cache/pip/wheels/37/5e/9e/76d7430e116b7cab0016fbabb26b896daae1946a3f7dea9915
Successfully built pycuda pytools


In [ ]:
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np
import math

In [ ]:
class MinimumEnergyControl:
    def __init__(self, x_des, x_0, step=50, dt=0.05, damping=False):

        ## gravity, criterion: moon
        gravity = -1.62     # N/kg

        ## no drag or something disturb movement
        if not damping:
            ## A
            self.state_transition_matrix = \
                np.array([[ 1, 0, 0,dt, 0, 0],
                          [ 0, 1, 0, 0,dt, 0],
                          [ 0, 0, 1, 0, 0,dt],
                          [ 0, 0, 0, 1, 0, 0],
                          [ 0, 0, 0, 0, 1, 0],
                          [ 0, 0, 0, 0, 0, 1]])

            ## B
            input_matrix = \
                np.array([[dt*dt/2,      0,      0],
                          [      0,dt*dt/2,      0],
                          [      0,      0,dt*dt/2],
                          [     dt,      0,      0],
                          [      0,     dt,      0],
                          [      0,      0,     dt]])
            
            self.input_matrix = \
                gpuarray.to_gpu(np.float32(np.array([0.5*dt*dt, dt])))

            ## g
            gravity_matrix = \
                np.array([[              0],
                          [              0],
                          [gravity*dt*dt/2],
                          [              0],
                          [              0],
                          [     gravity*dt]])
                
            self.gravity_matrix = \
                gpuarray.to_gpu(np.float32(np.array([0.5*gravity*dt*dt, gravity*dt])))

        ## drag or something exist...
        else:
            pass

        ## desired state: x_des
        self.x_des = gpuarray.to_gpu(np.float32(x_des))

        ## initial state: x_0
        self.x_0 = gpuarray.to_gpu(np.float32(x_0))

        self.dt = np.float32(dt)

        self.step = step

        ## weight
        self.rho = 3

        self.rho_matrix = \
            gpuarray.to_gpu(np.float32(self.rho * np.identity(3*self.step).reshape(3*3*self.step*self.step)))
        
        ## solution!!!
        self.u = gpuarray.to_gpu(np.float32(np.zeros((3*self.step,1))))

        ## G, gram_G, Q
        self.G = gpuarray.to_gpu(np.float32(np.zeros((3*self.step,6)).reshape(6*3*self.step)))
        self.gram_G = gpuarray.to_gpu(np.float32(np.zeros((3*self.step,3*self.step)).reshape(3*3*self.step*self.step)))
        self.Q = gpuarray.to_gpu(np.float32(np.zeros((6,1))))
        self.G_C = gpuarray.to_gpu(np.float32(np.zeros((150,1))))
        self.gradient = gpuarray.to_gpu(np.float32(np.zeros((150,1))))

        ## TPB: thread_per_block, BPG: block_per_grid
        self.TPB, self.iteration = self.optimal_size(3*self.step) 

    def run(self):
        self.get_gradient(self.gram_G,
                              self.u,
                              self.G_C,          
                              self.iteration,
                              self.gradient,
                              block=(self.TPB,1,1),
                              grid=(3*self.step,1,1))

    def optimal_size(self, n):
        thread_per_block = int(math.sqrt(n/2))

        iteration = int(n / thread_per_block) + 1

        return thread_per_block, np.int32(iteration)

    def define_object_function_at_kernel(self):
        self.ker_function()

        self.get_G_matrix(self.input_matrix, self.dt, self.G, block=(6,1,1), grid=(self.step,1,1))
        self.get_Q_matrix(self.gravity_matrix, self.dt, self.Q, block=(self.step,1,1), grid=(2,1,1))
        self.get_G_gram_matrix(self.G, self.rho_matrix, self.gram_G, block=(3,1,1), grid=(self.step,self.step,1))
        self.get_G_C_matrix(self.G, self.x_des, self.x_0, self.Q, self.G_C, block=(3,1,1), grid=(self.step,1,1))

    def ker_function(self):
        ## We'll gonna do 150 x 150 @ 150 x 1
        ## block=(thread_per_block,1,1), grid=(3*self.step,1,1)
        get_gradient_ker_function = \
        """
        #define tx (threadIdx.x)
        #define bx (blockIdx.x)
        #define bs (blockDim.x)
        #define gs (gridDim.x)

        __global__ void get_gradient(float* matrix, float* vector1, float* vector2, int iteration, float* result) {

            __shared__ float result_jerk[1000];

            result_jerk[tx] = 0.0;

            for (int i = 0; i < iteration; i++) {
                int index1 = i + tx * iteration;
                int index2 = index1 + bx * 150;

                if (index1 < gs) {
                    result_jerk[tx] += matrix[index2] * vector1[index1];
                }
                else {
                    result_jerk[1000-tx] = 0;
                }
            }

            __syncthreads();

            if (tx == 0) {
                for (int j = 0; j < bs; j++) {
                    result[bx] += result_jerk[j];
                }

                result[bx] -= vector2[bx];
            }
            else {
                result_jerk[1000-tx] = 0;
            }

            __syncthreads();
        }
        """
        get_gradient_ker = SourceModule(get_gradient_ker_function)

        ## block=(6,1,1), grid=(self.step,1,1)
        get_G_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)
        #define step (gridDim.x)

        __global__ void get_G_matrix(float* input_matrix, float dt, float* G) {
            // 6: DOF, 18: DOF*axis
            int index = tx + (tx%3) * 6 + bx * 18;

            if (tx < 3) {
                float value;
                value = input_matrix[0] + (step - bx - 1) * input_matrix[1];

                G[index] = value;
            }
            else {
                G[index] = dt;
            }

            __syncthreads();
        }
        """
        get_G_matrix_ker = SourceModule(get_G_matrix_ker_function)

        ## block=(self.step,1,1), grid=(2,1,1)
        get_Q_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)
        #define step (blockDim.x)

        __global__ void get_Q_matrix(float* gravity, float dt, float* Q) {
            
            __shared__ float value[50];
            
            if (bx == 0) {
                value[tx] = gravity[0] + (tx * dt) * gravity[1];
            }
            else {
                value[tx] = gravity[1];
            }

            __syncthreads();

            if (bx == 0) {
                if(tx == 0) {
                    for (int i = 0; i < step; i++) {
                        Q[2] += value[i];
                    }
                }
            }
            else {
                if(tx == 0) {
                    for (int i = 0; i < step; i++) {
                        Q[5] += value[i];
                    }
                }
            }

            __syncthreads();
        }
        """
        get_Q_matrix_ker = SourceModule(get_Q_matrix_ker_function)

        ## block=(3,1,1), grid=(self.step,self.step,1)
        get_G_gram_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define by (blockIdx.y)
        #define tx (threadIdx.x)
        #define step (gridDim.x)

        __global__ void get_G_gram_matrix(float* G, float* rho_matrix, float* gram_G) {
            // 9: axis, 151: axis*step+1, 450: axis*axis*step
            int index1 = tx * 151 + bx * 3 + by * 450;

            // 7: DOF+1, 18: DOF*axis
            int index2 = tx * 7 + bx * 18;
                
            float value;
            value = G[index2] * G[index2] + G[index2+3] * G[index2+3];

            gram_G[index1] = value; 

            __syncthreads();

            gram_G[index1] += rho_matrix[index1];

            __syncthreads();
        }
        """
        get_G_gram_matrix_ker = SourceModule(get_G_gram_matrix_ker_function)

        ## block=(3,1,1), grid=(self.step,1,1)
        get_G_C_matrix_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)

        __global__ void get_G_C_matrix(float* G, float* x_des, float* x_0, float* Q, float* G_C) {
            // C first in each block
            __shared__ float C[6];

            C[tx] = x_des[tx] - Q[tx] - x_0[tx];
            C[tx+3] = x_des[tx+3] - Q[tx+3] - x_0[tx+3];

            __syncthreads();

            // G_C Next
            int index1 = tx * 7 + bx * 18;
            int index2 = tx + bx * 3;

            float value;
            value = G[index1] * C[tx] + G[index1+3] * C[tx+3];

            __syncthreads();

            G_C[index2] = value;

            __syncthreads();
        }
        """
        get_G_C_matrix_ker = SourceModule(get_G_C_matrix_ker_function)

        self.get_G_matrix = get_G_matrix_ker.get_function("get_G_matrix")
        self.get_Q_matrix = get_Q_matrix_ker.get_function("get_Q_matrix")
        self.get_G_gram_matrix = get_G_gram_matrix_ker.get_function("get_G_gram_matrix")
        self.get_G_C_matrix = get_G_C_matrix_ker.get_function("get_G_C_matrix")
        self.get_gradient = get_gradient_ker.get_function("get_gradient")

In [ ]:
class Optimizer:
    def __init__(self, length):
        self.length = length
        self.kernel_function()

    def run(self, theta, gradient, learning_rate):
        ## theta, gradient: gpuarray type variable
        self.basic_optimizer(theta,
                             gradient,
                             learning_rate,
                             block=(self.length,1,1),
                             grid=(1,1,1))

    def kernel_function(self):
        ## block=(length,1,1), grid=(1,1,1)
        basic_optimizer_ker_function = \
        """
        #define x (threadIdx.x)

        __global__ void basic_optimizer(float* theta, float* gradient, float learning_rate) {
            theta[x] -= gradient[x] * learning_rate;

            __syncthreads();
        }
        """
        basic_optimizer_ker = SourceModule(basic_optimizer_ker_function)

        self.basic_optimizer = basic_optimizer_ker.get_function("basic_optimizer")

In [ ]:
class MinimumEnergyControlSolver:
    def __init__(self, x_des, x_0):
        ## initialize MEC
        self.MEC = MinimumEnergyControl(x_des, x_0)
        self.MEC.define_object_function_at_kernel()

        ## initialize optimizer
        self.optimizer = Optimizer(3*self.MEC.step)

    def solve(self):
        for i in range(100):
            ## get_gradient
            self.MEC.run()

            ## optimize
            self.optimizer.run(self.MEC.u, self.MEC.gradient, np.float32(1e-4))

## Test

In [ ]:
## destination
x_des = np.array([0,0,0,0,0,0])

## initial point
x_0 = np.array([100,0,-1500,-10,0,80])

MECS = MinimumEnergyControlSolver(x_des, x_0)

In [ ]:
MECS.solve()

MECS.MEC.u[:12]

array([[-26.1415  ],
       [  0.      ],
       [393.5056  ],
       [-24.192976],
       [  0.      ],
       [364.17877 ],
       [-22.244417],
       [  0.      ],
       [334.852   ],
       [-20.29588 ],
       [  0.      ],
       [305.525   ]], dtype=float32)

Bravo!

## Constraint

In [ ]:
class ConstraintsForInput:
    def __init__(self, problem, upper_boundary, downer_boundary):
        ## ex> MEC(minimum energy control)
        self.problem = problem

        self.upper_boundary = np.float32(upper_boundary)
        self.downer_boundary = np.float32(downer_boundary)

        self.kernel_function()

    def projection(self):
        self.project_function(self.problem.u,
                              self.upper_boundary,
                              self.downer_boundary,
                              block=(3,1,1),
                              grid=(self.problem.step))

    def kernel_function(self):
        ## block=(3,1,1), grid=(problem.step,1,1)
        projection_ker_function = \
        """
        #define bx (blockIdx.x)
        #define tx (threadIdx.x)

        __global__ void projection(float* theta, float upper_boundary, float downer_boundary) {
            __shared__ float u[3];
            __shared__ float norm[1];
            float value;

            int index = tx + bx * 3;

            u[tx] = theta[index] * theta[index];

            __syncthreads();

            if (tx == 0) {
                norm[0] = get_norm(u[0], u[1], u[2]);
            } 

            __syncthreads();

            if ((norm[0] > downer_boundary) || (norm[0] < upper_boundary)) {
                value = theta[index];
            }
            else {
                value = theta[index] * upper_boundary / norm[0];
            }

            __syncthreads();

            theta[index] = value;
        }

        __device__ float get_norm(float x, float y, float z) {
            float value;
            float norm;

            value = x * x + y * y + z * z;
            norm = square_root(value);

            return norm;    
        }

        __device__ float square_root(float value) {
            float s = 0;
            float t = 0;

            s = value / 2;

            for (;s != t;) {
                t = s;
                s = ((value/t) + t) / 2;
            }

            return s;
        }
        """
        projection_ker = SourceModule(projection_ker_function)

        self.project_function = projection_ker.get_function("projection")

### Redefine Solver

In [ ]:
class MinimumEnergyControlSolver:
    def __init__(self, x_des, x_0, upper_boundary, downer_boundary):
        ## initialize MEC(minimum energy control)
        self.MEC = MinimumEnergyControl(x_des, x_0)
        self.MEC.define_object_function_at_kernel()

        ## initialize optimizer
        self.optimizer = Optimizer(3*self.MEC.step)

        ## constraint
        self.upper_boundary = upper_boundary
        self.donwer_boundary = downer_boundary

        self.constraint = ConstraintsForInput(self.MEC, self.upper_boundary, self.downer_boundary)

    def solve(self):
        for i in range(100):
            ## get_gradient
            self.MEC.run()

            ## optimize
            self.optimizer.run(self.MEC.u, self.MEC.gradient, np.float32(1e-4))

            ## constraint
            self.constraint.projection()

In [ ]:
## upper boundary: 5.8, downer boundary: 0.0

## destination
x_des = np.array([0,0,0,0,0,0])

## initial point
x_0 = np.array([100,0,-1500,-10,0,80])

## constraints
upper_boundary = 5.8
downer_boundary = 0.0

MECS = MinimumEnergyControlSolver(x_des, x_0, upper_boundary, downer_boundary)

In [ ]:
MECS.solve()

MECS.MEC.u[:12]

## Not tested yet